In [1]:
include("/Users/meesvandartel/Desktop/Coursework/CGT/DeepEWA/FastEWA.jl")
using Flux, .fEWA, Random, ProgressMeter, CSV, DataFrames



In [2]:
train = CSV.read("/Users/meesvandartel/Desktop/Coursework/CGT/localdata_DeepEWA/coord_train.csv", DataFrame)
test = CSV.read("/Users/meesvandartel/Desktop/Coursework/CGT/localdata_DeepEWA/coord_test.csv", DataFrame)


Row,x_test,y_test
,String,String
1,"[0.4151, 0.047, 0.4423, 3.070990045496511]","([2, 2], Array{Float64}[[0.0, 1.0], [0.0, 1.0]], Array{Float64}[[0.4423, 4.0], [0.4423, 4.0]], true)"
2,"[0.9699, 0.2836, 0.3718, 4.599741011213829]","([1, 1], Array{Float64}[[0.001998001998001998, 0.998001998001998], [0.001998001998001998, 0.998001998001998]], Array{Float64}[[5.0, 0.3718], [5.0, 0.3718]], true)"
3,"[0.7587, 0.0792, 0.2169, 40.406877272619965]","([2, 2], Array{Float64}[[0.0, 1.0], [0.0, 1.0]], Array{Float64}[[0.2169, 4.0], [0.2169, 4.0]], true)"
4,"[0.267, 0.3875, 0.9499, 10.601547701127542]","([1, 1], Array{Float64}[[0.001998001998001998, 0.998001998001998], [0.001998001998001998, 0.998001998001998]], Array{Float64}[[5.0, 0.9499], [5.0, 0.9499]], true)"
5,"[0.9271, 0.3051, 0.418, 22.737146574588383]","([1, 2], Array{Float64}[[0.5004995004995005, 0.49950049950049946], [0.4995004995004995, 0.5004995004995005]], Array{Float64}[[1.0, 1.672], [2.09, 1.0]], false)"
6,"[0.5965, 0.4295, 0.6287, 1.8130309449601565]","([2, 2], Array{Float64}[[0.0, 1.0], [0.0, 1.0]], Array{Float64}[[0.6287, 4.0], [0.6287, 4.0]], true)"
7,"[0.5835, 0.9268, 0.7372, 1.2712491503214047]","([1, 1], Array{Float64}[[0.001998001998001998, 0.998001998001998], [0.001998001998001998, 0.998001998001998]], Array{Float64}[[5.0, 0.7372], [5.0, 0.7372]], true)"
8,"[0.5924, 0.3284, 0.7447, 6.705982242240201]","([1, 1], Array{Float64}[[0.001998001998001998, 0.998001998001998], [0.001998001998001998, 0.998001998001998]], Array{Float64}[[5.0, 0.7447], [5.0, 0.7447]], true)"
9,"[0.4066, 0.1341, 0.6478, 52.878667676447755]","([2, 1], Array{Float64}[[0.4995004995004995, 0.5004995004995005], [0.5004995004995005, 0.49950049950049946]], Array{Float64}[[3.2390000000000003, 1.0], [1.0, 2.5912]], false)"


In [ ]:
DeepEWA= Chain(
    Dense(4 => 6, relu),
    Dense(6 => 6, relu),
    Dense(6 => 2))

out1 = DeepEWA(x_train)
probs1 = softmax(out1)

target = Flux.onehotbatch(y_train, ["true", "false"])
loader = Flux.DataLoader((x_train, target), batchsize=64, shuffle=true);

opt_state = Flux.setup(Flux.Adam(0.01), DeepEWA)

losses=[]
@showprogress for epoch in 1:1000
    for xy in loader
        x,y = xy
        loss, grads = Flux.withgradient(DeepEWA) do m
            y_hat = m(x)
            Flux.logitcrossentropy(y_hat, y)
        end
        Flux.update!(opt_state, model, grads[1])
        push!(losses, loss)
    end
end

opt_state

In [ ]:
out2 = model(x_train)
probs2 = softmax(out2)
mean((probs2[1,:].>0.5).==truth)

In [ ]:
using Plots  # to draw the above figure

p_true = scatter(x_train[1,:], x_train[2,:], zcolor=truth, title="True classification", legend=false)
p_raw =  scatter(x_train[1,:], x_train[2,:], zcolor=probs1[1,:], title="Untrained network", label="", clims=(0,1))
p_done = scatter(x_train[1,:], x_train[2,:], zcolor=probs2[1,:], title="Trained network", legend=false)

plot(p_true, p_raw, p_done, layout=(1,3), size=(1000,330))


In [ ]:
plot(losses; xaxis=(:log10, "iteration"),
    yaxis="loss", label="per batch")
n = length(loader)
plot!(n:n:length(losses), mean.(Iterators.partition(losses, n)),
    label="epoch mean", dpi=200)
